In [28]:
import pandas as pd
import random 
import json

experiment_folder = "second_experiment"

num_sentences_tech = 204
num_sentences_knowledge = 104

tech_file_path = f'data/experiments/{experiment_folder}/{num_sentences_tech}_tech_sentences_with_prompts.csv'
house_file_path = f'data/experiments/{experiment_folder}/{num_sentences_knowledge}_house_sentences_with_prompts.csv'

tech_sentences_df = pd.read_csv(tech_file_path)
house_sentences_df = pd.read_csv(house_file_path)

num_rewrites_tech = 423
num_rewrites_house = 221

with open(f"data/experiments/{experiment_folder}/{num_rewrites_tech}_tech_rewrites.json", 'r') as f:
    tech_rewrites = json.load(f)

with open(f"data/experiments/{experiment_folder}/{num_rewrites_house}_house_rewrites.json", 'r') as f:
    house_rewrites = json.load(f)

tech_sentences_df

,occupation_name,skills,knowledge,prompt,results,original_sentences
0,blockchain architect,['design information system'],[],"Take this sentence from a Job Posting.\n\n""* P...",* Passionate about @@designing information sys...,"""* Passionate about @@building an innovative c..."
1,system architect,[],['COBOL'],"Take this sentence from a Job Posting.\n\n""• B...",• Basic ##COBOL##.,"""• Basic ##UNIX scripting## ."""
2,application configurator,['use software libraries'],[],"Take this sentence from a Job Posting.\n\n""@@C...",@@Use software libraries@@ to analyze and synt...,"""@@Capture analyze and synthesize research fin..."
3,digital games tester,['develop test suite'],['debugging tools'],"Take this sentence from a Job Posting.\n\n""Und...",Understand ##debugging tools## from both a man...,"""Understand ##endpoint protect## from both a m..."
4,digital games developer,['develop software prototype'],[],"Take this sentence from a Job Posting.\n\n""@@B...",@@Develop software prototype@@ to meet endpoin...,"""@@Build secure system solutions@@ to meet end..."
...,...,...,...,...,...,...
199,software tester,['debug software'],[],"Take this sentence from a Job Posting.\n\n""As ...",As an experienced member of our Software Engin...,"""As an experienced member of our Software Engi..."
200,digital games tester,"['use scripting programming', 'address problem...",[],"Take this sentence from a Job Posting.\n\n""Str...",Strong @@use scripting programming@@ and the a...,"""Strong @@communication skills@@ and the abili..."
201,test analyst,[],"['visual presentation techniques', 'decision s...","Take this sentence from a Job Posting.\n\n""You...",You will enjoy working with a modern tech stac...,"""You will enjoy working with a modern tech sta..."
202,digital games developer,[],"['Ruby (computer programming)', 'Lisp', 'digit...","Take this sentence from a Job Posting.\n\n""* D...",* Deep understanding of core areas like ##Ruby...,"""* Deep understanding of core ##AWS## services..."


In [29]:
def convert_to_bio_tagged_sentences(sentence):
    words = sentence.replace('##', ' ## ').replace('@@', ' @@ ').split()
    tagged_words = []
    
    skill_tag = 'O'  
    knowledge_tag = 'O'  

    for word in words:
        if word == '@@':
            if skill_tag == 'O':
                skill_tag = 'B-Skill'  
            else:
                skill_tag = 'O'  
            continue  
        
        if word == '##':
            if knowledge_tag == 'O':
                knowledge_tag = 'B-Knowledge'  
            else:
                knowledge_tag = 'O'  
            continue  
        
        tagged_words.append((word, skill_tag, knowledge_tag))

        if skill_tag == 'B-Skill':
            skill_tag = 'I-Skill'
        if knowledge_tag == 'B-Knowledge':
            knowledge_tag = 'I-Knowledge'
    
    return tagged_words

In [30]:
def split_sentences_to_bio(sentences_to_split):
    splitted_sentences = []

    for sentence_to_split in sentences_to_split:
        splitted_sentences.append(convert_to_bio_tagged_sentences(sentence_to_split))
        
    return splitted_sentences

In [31]:
def append_sentences_to_conll(filename, sentences_to_append):
    with open(filename, 'a') as file:
        for sentence in sentences_to_append:
            file.write("\n")  
            for word, skill_tag, knowledge_tag in sentence:
                file.write(f"{word}\t{skill_tag}\t{knowledge_tag}\n")


In [32]:
bio_splitted_sentences = split_sentences_to_bio(tech_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [33]:
bio_splitted_sentences = split_sentences_to_bio(house_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)

In [34]:
bio_splitted_sentences = split_sentences_to_bio(tech_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [35]:
bio_splitted_sentences = split_sentences_to_bio(house_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)